# Example: using OpenMM to run a simulation with a NequIP ML potential

You can run this example directly in your browser: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sef43/openmm-ml/blob/nequip/examples/nequip/run_nequip.ipynb)

## Install Conda

[Conda](https://docs.conda.io/) is a package and environment manager. On Google Colab, Conda is installed with [conda-colab](https://github.com/jaimergp/condacolab). On your computer, you should follow these [installation instructions](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).

⚠️ Do not use conda-colab on your computer!

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge() # use mamba on colab because it is faster than conda

## Install software

First install everything we can from [conda-forge](https://conda-forge.org/).
Then use pip.

⚠️ The installation might take up to 10 min!

In [ ]:
#https://github.com/openmm/openmm-torch/issues/88
%env CONDA_OVERRIDE_CUDA=11.2 
!mamba install -c conda-forge openmm-torch

!pip install git+https://github.com/mir-group/nequip@develop
!pip install git+https://github.com/sef43/openmm-ml@nequip

## Get the files we need to run the example

In [ ]:
!wget https://raw.githubusercontent.com/sef43/openmm-ml/nequip/examples/nequip/toluene.pdb
!wget https://raw.githubusercontent.com/sef43/openmm-ml/nequip/examples/nequip/example_model_deployed.pth

## Run simulation

In [2]:
import openmm
import openmm.app as app
import openmm.unit as unit
from openmmml import MLPotential
from sys import stdout

"""
Uses a deployed trained NequIP model, toluene example:
nequip-train configs/example.yaml
nequip-deploy build --train-dir path/to/training/session/ example_model_deployed.pth
"""

# load toluene structure
pdb = app.PDBFile("toluene.pdb")

# create a System with NequIP MLP

# need to specify the unit conversion factors from the NequIP model units to OpenMM units.
# distance: model is in Angstrom, OpenMM is in nanometers
A_to_nm = 0.1
# energy: model is in kcal/mol, OpenMM is in kJ/mol
kcal_to_kJ_per_mol = 4.184

potential = MLPotential('nequip', model_path='example_model_deployed.pth',
                        distance_to_nm=A_to_nm,
                        energy_to_kJ_per_mol=kcal_to_kJ_per_mol)

system = potential.createSystem(pdb.topology)

# run langevin dynamics at 300K for 1000 steps
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 10.0/unit.picoseconds, 1.0*unit.femtosecond)
simulation=app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.reporters.append(app.PDBReporter('output.pdb', 100))
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True))

simulation.step(1000)

# Minimize the energy
simulation.minimizeEnergy()
energy=simulation.context.getState(getEnergy=True).getPotentialEnergy()
print(energy, energy.in_units_of(unit.kilocalorie_per_mole))

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-710472.8125,204.91816875199336
200,-710471.125,278.916390104101
300,-710463.3125,277.04602043230767
400,-710468.125,238.63950096559105
500,-710464.75,240.2583527290122
600,-710459.0625,320.5201903960295
700,-710446.9375,340.2834476858224
800,-710440.625,391.44641207719144
900,-710463.0,396.8090709289139
1000,-710445.875,259.7874673879626
-710513.4375 kJ/mol -169816.78716539196 kcal/mol
